# 🏦 Verificación del Entorno - Curso SQL Banking Analytics

## 📋 Lista de Verificación Pre-Curso

Este notebook verifica que todas las dependencias y conexiones estén funcionando correctamente para el curso de SQL orientado a la banca.

### ✅ Objetivos de Verificación:
1. **Dependencias instaladas** correctamente
2. **Conexión a base de datos** SQLite funcionando
3. **Tablas bancarias** accesibles
4. **Consultas básicas** operativas
5. **Entorno listo** para el curso

---

## 🎯 Pre-requisitos del Curso

- Python 3.8+
- Jupyter Lab/Notebook
- SQLite3
- Pandas, SQLAlchemy
- Base de datos `banking_core.db`

## 📦 1. Instalación de Dependencias Necesarias

Verificamos e instalamos los paquetes requeridos para el análisis bancario con SQL.

In [ ]:
# Verificación e instalación de dependencias
import sys
import subprocess
import pkg_resources

def verificar_paquete(paquete):
    """Verifica si un paquete está instalado"""
    try:
        pkg_resources.get_distribution(paquete)
        return True
    except pkg_resources.DistributionNotFound:
        return False

def instalar_paquete(paquete):
    """Instala un paquete usando pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", paquete])

# Lista de paquetes requeridos
paquetes_requeridos = [
    'pandas',
    'numpy', 
    'matplotlib',
    'seaborn',
    'sqlalchemy',
    'plotly',
    'ipywidgets'
]

print("🔍 Verificando dependencias...")
print("=" * 50)

for paquete in paquetes_requeridos:
    if verificar_paquete(paquete):
        print(f"✅ {paquete} - INSTALADO")
    else:
        print(f"❌ {paquete} - NO ENCONTRADO")
        try:
            print(f"📦 Instalando {paquete}...")
            instalar_paquete(paquete)
            print(f"✅ {paquete} - INSTALADO EXITOSAMENTE")
        except Exception as e:
            print(f"⚠️  Error instalando {paquete}: {e}")

print("\n🎉 Verificación de dependencias completada!")

## 🔗 2. Conexión a la Base de Datos SQL

Configuramos la conexión a nuestra base de datos SQLite que contiene los datos bancarios de ejemplo.

In [ ]:
# Importar librerías necesarias
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import os
from pathlib import Path

# Configuración de la base de datos
DB_PATH = "../data/banking_core.db"

print("🔗 Configurando conexión a la base de datos...")
print("=" * 50)

# Verificar que el archivo de base de datos existe
if os.path.exists(DB_PATH):
    print(f"✅ Base de datos encontrada: {DB_PATH}")
    
    # Crear conexión SQLite
    try:
        conn_sqlite = sqlite3.connect(DB_PATH)
        print("✅ Conexión SQLite establecida")
        
        # Crear motor SQLAlchemy (útil para pandas)
        engine = create_engine(f'sqlite:///{DB_PATH}')
        print("✅ Motor SQLAlchemy creado")
        
        print("\n🎉 Conexión a base de datos exitosa!")
        
    except Exception as e:
        print(f"❌ Error conectando a la base de datos: {e}")
        
else:
    print(f"❌ No se encontró la base de datos en: {DB_PATH}")
    print("💡 Tip: Verifica que la ruta sea correcta o ejecuta el script setup_environment.py")

## ✅ 3. Verificación de la Conexión

Ejecutamos una consulta simple para confirmar que la conexión funciona correctamente.

In [ ]:
# Verificar conexión con una consulta simple
print("🔍 Verificando conexión con consulta de prueba...")
print("=" * 50)

try:
    # Consulta simple para verificar SQLite
    cursor = conn_sqlite.cursor()
    cursor.execute("SELECT sqlite_version();")
    version = cursor.fetchone()
    print(f"✅ SQLite versión: {version[0]}")
    
    # Verificar que podemos ejecutar consultas básicas
    cursor.execute("SELECT 'Conexión exitosa' as mensaje, datetime('now') as timestamp;")
    resultado = cursor.fetchone()
    print(f"✅ Mensaje: {resultado[0]}")
    print(f"✅ Timestamp: {resultado[1]}")
    
    print("\n🎉 Verificación de conexión completada exitosamente!")
    
except Exception as e:
    print(f"❌ Error en la verificación: {e}")
    print("💡 Tip: Revisa la conexión y la base de datos")

## 📊 4. Exploración de las Tablas Disponibles

Identificamos todas las tablas del sistema bancario y su estructura.

In [ ]:
# Explorar tablas disponibles en la base de datos
print("📊 Explorando estructura de la base de datos...")
print("=" * 50)

try:
    # Obtener lista de todas las tablas
    cursor.execute("""
        SELECT name 
        FROM sqlite_master 
        WHERE type='table' 
        AND name NOT LIKE 'sqlite_%'
        ORDER BY name;
    """)
    
    tablas = cursor.fetchall()
    
    print(f"🏦 TABLAS DEL SISTEMA BANCARIO ({len(tablas)} encontradas):")
    print("-" * 50)
    
    for i, tabla in enumerate(tablas, 1):
        tabla_nombre = tabla[0]
        
        # Obtener información sobre cada tabla
        cursor.execute(f"SELECT COUNT(*) FROM {tabla_nombre}")
        num_registros = cursor.fetchone()[0]
        
        cursor.execute(f"PRAGMA table_info({tabla_nombre})")
        columnas = cursor.fetchall()
        num_columnas = len(columnas)
        
        print(f"{i:2}. 📋 {tabla_nombre}")
        print(f"    - Registros: {num_registros:,}")
        print(f"    - Columnas: {num_columnas}")
        print()
    
    print("🎉 Exploración de tablas completada!")
    
except Exception as e:
    print(f"❌ Error explorando tablas: {e}")

## 🏦 5. Consulta Básica de Datos Bancarios

Realizamos consultas de ejemplo para verificar que los datos bancarios sean accesibles.

In [ ]:
# Consultas básicas de verificación en datos bancarios
print("🏦 Ejecutando consultas de verificación en datos bancarios...")
print("=" * 60)

# 1. Verificar tabla de clientes
print("1️⃣ TABLA CLIENTES:")
try:
    query_clientes = """
        SELECT 
            COUNT(*) as total_clientes,
            COUNT(DISTINCT segmento_cliente) as segmentos,
            MIN(fecha_vinculacion) as cliente_mas_antiguo,
            MAX(fecha_vinculacion) as cliente_mas_reciente
        FROM clientes 
        WHERE estado = 'ACTIVO'
    """
    
    resultado = pd.read_sql_query(query_clientes, conn_sqlite)
    print(f"   ✅ Clientes activos: {resultado['total_clientes'].iloc[0]:,}")
    print(f"   ✅ Segmentos: {resultado['segmentos'].iloc[0]}")
    print(f"   ✅ Cliente más antiguo: {resultado['cliente_mas_antiguo'].iloc[0]}")
    print(f"   ✅ Cliente más reciente: {resultado['cliente_mas_reciente'].iloc[0]}")
    
except Exception as e:
    print(f"   ❌ Error en clientes: {e}")

print()

# 2. Verificar tabla de cuentas
print("2️⃣ TABLA CUENTAS:")
try:
    query_cuentas = """
        SELECT 
            COUNT(*) as total_cuentas,
            SUM(saldo_actual) as saldo_total,
            AVG(saldo_actual) as saldo_promedio,
            COUNT(DISTINCT tipo_cuenta) as tipos_cuenta
        FROM cuentas 
        WHERE estado = 'ACTIVA'
    """
    
    resultado = pd.read_sql_query(query_cuentas, conn_sqlite)
    print(f"   ✅ Cuentas activas: {resultado['total_cuentas'].iloc[0]:,}")
    print(f"   ✅ Saldo total: ${resultado['saldo_total'].iloc[0]:,.2f}")
    print(f"   ✅ Saldo promedio: ${resultado['saldo_promedio'].iloc[0]:,.2f}")
    print(f"   ✅ Tipos de cuenta: {resultado['tipos_cuenta'].iloc[0]}")
    
except Exception as e:
    print(f"   ❌ Error en cuentas: {e}")

print()

# 3. Verificar tabla de transacciones
print("3️⃣ TABLA TRANSACCIONES:")
try:
    query_transacciones = """
        SELECT 
            COUNT(*) as total_transacciones,
            SUM(CASE WHEN monto > 0 THEN monto ELSE 0 END) as ingresos_total,
            SUM(CASE WHEN monto < 0 THEN ABS(monto) ELSE 0 END) as egresos_total,
            COUNT(DISTINCT tipo_transaccion) as tipos_transaccion
        FROM transacciones 
        WHERE fecha_transaccion >= date('now', '-30 days')
    """
    
    resultado = pd.read_sql_query(query_transacciones, conn_sqlite)
    print(f"   ✅ Transacciones (últimos 30 días): {resultado['total_transacciones'].iloc[0]:,}")
    print(f"   ✅ Ingresos totales: ${resultado['ingresos_total'].iloc[0]:,.2f}")
    print(f"   ✅ Egresos totales: ${resultado['egresos_total'].iloc[0]:,.2f}")
    print(f"   ✅ Tipos de transacción: {resultado['tipos_transaccion'].iloc[0]}")
    
except Exception as e:
    print(f"   ❌ Error en transacciones: {e}")

print("\n🎉 Consultas básicas completadas exitosamente!")

## ⚠️ 6. Manejo de Errores de Conexión

Incluimos ejemplos de manejo de errores comunes y sus soluciones.

In [ ]:
# Función para manejo robusto de conexiones y errores
def conectar_base_datos_segura(ruta_db):
    """
    Función robusta para conectar a la base de datos con manejo de errores
    """
    try:
        # Verificar que el archivo existe
        if not os.path.exists(ruta_db):
            raise FileNotFoundError(f"Base de datos no encontrada: {ruta_db}")
        
        # Intentar conexión
        conn = sqlite3.connect(ruta_db)
        
        # Verificar que la conexión funciona
        cursor = conn.cursor()
        cursor.execute("SELECT 1")
        cursor.fetchone()
        
        print(f"✅ Conexión exitosa a: {ruta_db}")
        return conn, cursor
        
    except FileNotFoundError as e:
        print(f"❌ Error: Archivo no encontrado - {e}")
        print("💡 Solución: Verifica la ruta de la base de datos")
        return None, None
        
    except sqlite3.Error as e:
        print(f"❌ Error SQLite: {e}")
        print("💡 Solución: Verifica que el archivo no esté corrupto")
        return None, None
        
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        print("💡 Solución: Contacta al administrador del sistema")
        return None, None

def ejecutar_consulta_segura(cursor, query, descripcion="Consulta"):
    """
    Ejecuta una consulta SQL con manejo de errores
    """
    try:
        cursor.execute(query)
        resultado = cursor.fetchall()
        print(f"✅ {descripcion} ejecutada exitosamente")
        return resultado
        
    except sqlite3.Error as e:
        print(f"❌ Error en {descripcion}: {e}")
        print("💡 Solución: Revisa la sintaxis SQL")
        return None
        
    except Exception as e:
        print(f"❌ Error inesperado en {descripcion}: {e}")
        return None

# Ejemplo de uso del manejo de errores
print("🛡️ Probando manejo robusto de errores...")
print("=" * 50)

# Probar conexión con ruta incorrecta
conn_test, cursor_test = conectar_base_datos_segura("ruta/inexistente.db")

# Probar conexión con ruta correcta
conn_test, cursor_test = conectar_base_datos_segura(DB_PATH)

if cursor_test:
    # Probar consulta incorrecta
    ejecutar_consulta_segura(cursor_test, "SELECT * FROM tabla_inexistente", "Consulta de prueba (error esperado)")
    
    # Probar consulta correcta
    ejecutar_consulta_segura(cursor_test, "SELECT COUNT(*) FROM clientes", "Conteo de clientes")
    
    conn_test.close()

print("\n🎉 Pruebas de manejo de errores completadas!")

## 🎯 Resumen de Verificación

### ✅ Lista de Verificación Completada:

1. **✅ Dependencias instaladas** - Pandas, SQLAlchemy, Matplotlib, etc.
2. **✅ Conexión SQLite funcionando** - Base de datos `banking_core.db` accesible  
3. **✅ Tablas bancarias verificadas** - Clientes, cuentas, transacciones, etc.
4. **✅ Consultas básicas operativas** - SELECT, COUNT, SUM funcionando
5. **✅ Manejo de errores implementado** - Funciones robustas de conexión

---

## 🚀 ¡Entorno Listo para el Curso!

Tu entorno está completamente configurado y listo para comenzar el **Curso de SQL Banking Analytics**. 

### 📚 Próximos Pasos:
- **Módulo 1**: Fundamentos SQL aplicados a banca
- **Módulo 2**: Agregaciones y KPIs bancarios  
- **Módulo 3**: Joins y relaciones entre tablas
- **Módulo 4**: Análisis de cartera crediticia
- **Módulo 5**: Reportes regulatorios

### 🔧 Herramientas Configuradas:
- **Base de datos**: SQLite con datos bancarios reales
- **Análisis**: Pandas + SQLAlchemy
- **Visualización**: Matplotlib + Seaborn + Plotly
- **Entorno**: Jupyter Lab listo para usar

¡Disfruta aprendiendo SQL para el sector bancario! 🏦📊

In [ ]:
# Cerrar conexiones de manera segura
try:
    if 'conn_sqlite' in locals() and conn_sqlite:
        conn_sqlite.close()
        print("✅ Conexión SQLite cerrada correctamente")
    
    if 'engine' in locals() and engine:
        engine.dispose()
        print("✅ Motor SQLAlchemy cerrado correctamente")
        
    print("\n🎉 ¡Verificación completada exitosamente!")
    print("🚀 Tu entorno está listo para el curso de SQL Banking Analytics")
    
except Exception as e:
    print(f"⚠️ Error cerrando conexiones: {e}")
    print("💡 No es crítico - las conexiones se cerrarán automáticamente")